In [1]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pickle

In [2]:
# Conseguir la lista de los equipos de La Liga 2015
lista_equipos = []
# Configuración del navegador
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)
# URL
base_url = "https://www.livefutbol.com/calendario/esp-primera-division-2015-2016-spieltag/38/"
driver.get(base_url)
boton_aceptar = WebDriverWait(driver, 20).until(
    EC.visibility_of_element_located((By.XPATH, '//a[@class="cmpboxbtn cmpboxbtnyes cmptxt_btn_yes"]'))
)
boton_aceptar.click()
print("Botón de aceptar clickeado")
time.sleep(10)
tabla = driver.find_element(By.CSS_SELECTOR, "table.standard_tabelle")
enlaces = tabla.find_elements(By.CSS_SELECTOR, 'a[href^="/equipos/"]')
# Recorrer cada enlace y extraer el nombre del equipo
for enlace in enlaces:
    href = enlace.get_attribute("href")  # Obtener la URL completa
    equipo = href.split("/equipos/")[1].split("/")[0]  # Extraer el nombre del equipo
    if equipo not in lista_equipos:
        lista_equipos.append(equipo)  # Añadir equipo a la lista si no está ya
    time.sleep(0.5)  # Dormir para evitar sobrecargar el servidor
driver.quit()


Botón de aceptar clickeado


In [5]:
# Crear URLs para cada equipo
plantilla_url = "https://www.livefutbol.com/equipos/{x}/2016/2/"
urls_por_equipo = {}
for equipo in lista_equipos:
    url = plantilla_url.format(x=equipo)
    urls_por_equipo[equipo] = url

In [7]:
# Conseguir una lista con todos los jugadores de cada equipo
jugadores = {}
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)
for x, i in urls_por_equipo.items():
    # URL
    driver.get(i)
    # Espera
    wait = WebDriverWait(driver, 10)
    # Clic en el botón de cookies
    try:
        boton_aceptar = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.XPATH, '//a[@class="cmpboxbtn cmpboxbtnyes cmptxt_btn_yes"]'))
        )
        boton_aceptar.click()
        print("Botón de aceptar clickeado")
    except:
        pass

    tabla = driver.find_element(By.CSS_SELECTOR, "table.standard_tabelle")
    filas = tabla.find_elements(By.CSS_SELECTOR, "tr")
    lista_jugadores = []

    for fila in filas:
        # Detectar encabezado de sección
        ths = fila.find_elements(By.TAG_NAME, "th")
        if ths:
            texto_encabezado = ths[0].text.strip().lower()
            if "entrenador" in texto_encabezado:
                print(f"Fin de plantilla detectado en sección: {texto_encabezado}")
                break  # Cortar el bucle al llegar al cuerpo técnico
            continue  # Saltar encabezado si es Portero, Defensa, etc.

        # Si es una fila de jugador, busca enlaces a fichas
        enlaces = fila.find_elements(By.CSS_SELECTOR, 'a[href^="/ficha_jugador/"]')
        for enlace in enlaces:
            href = enlace.get_attribute("href")
            jugador = href.split("/ficha_jugador/")[1].split("/")[0]
            lista_jugadores.append(jugador)
        
    jugadores[x] = lista_jugadores

    time.sleep(1)
driver.quit()

Botón de aceptar clickeado
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: segundo entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantilla detectado en sección: entrenador
Fin de plantill

In [22]:
# Crear una URL por cada jugador
plantilla_url_jugador = "https://www.livefutbol.com/ficha_jugador/{x}"
urls_por_jugador = {}
for equipo, plantilla in jugadores.items():
    urls_jugador = {}
    for jugador in plantilla:
        url = plantilla_url_jugador.format(x=jugador)
        urls_jugador[jugador] = url
    urls_por_jugador[equipo] = urls_jugador

In [ ]:
# Guardar a un archivo pickle con las url
with open("data/urls_por_jugador.pkl", "wb") as f:
    pickle.dump(urls_por_jugador, f)

In [3]:
# Abrir el archivo
with open("urls_por_jugador.pkl", "rb") as f:
    urls_por_jugador = pickle.load(f)

In [4]:
# Creamos listado_jugadores para no modificar el archivo
listado_jugadores = urls_por_jugador

In [5]:
# Obtenemos el historial de en que equipo ha jugado cada jugador
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

for equipo, plantilla in listado_jugadores.items():
    for jugador, url in plantilla.items():
        # Inicializar bio y historial para cada jugador
        bio = {"info": [], "historial": []}
        historial = []
        info = []
        
        driver.get(url)
        
        # Aceptar cookies si es necesario
        try:
            WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "cmpbntyestxt"))).click()
        except:
            pass

        # Aceptar cookies si es necesario (segundo intento)
        try:
            WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "cmpbntyestxt"))).click()
        except:
            pass

        # Obtener tabla del historial
        boxes = driver.find_elements(By.CLASS_NAME, "box")
        for box in boxes:
            try:
                h2 = box.find_element(By.TAG_NAME, "h2")
                if h2.text.strip() == "Trayectoria como jugador":
                    # Si el h2 coincide, busca la tabla dentro del mismo box
                    table = box.find_element(By.CLASS_NAME, "standard_tabelle")
                    break  # Salir del bucle una vez encontrada la tabla
            except:
                continue

        filas_historial = table.find_elements(By.CSS_SELECTOR, 'tr')
        info_historial = filas_historial[0:-1]
            
        # Recorrer cada enlace y extraer el historial del jugador
        for fila in info_historial:
            celdas = fila.find_elements(By.TAG_NAME, "td")
            fecha = celdas[0].text.strip()
            equipo_nombre = celdas[2].text.strip()
            if fecha != "":
                historial.append({
                        "fecha": fecha,
                        "equipo": equipo_nombre,
                    })
                bio["historial"] = historial
        
        # Obtener información del jugador
        try:
            nombre__deportivo_jugador = driver.find_element(By.CSS_SELECTOR, 'div.head h2[itemprop="name"]').text.strip()
            
            # Buscar la tabla con la información del jugador
            tabla_jugador = driver.find_element(By.CSS_SELECTOR, "table.standard_tabelle.yellow")
            filas_jugador = tabla_jugador.find_elements(By.CSS_SELECTOR, 'tr')
            celdas = filas_jugador[0].find_elements(By.TAG_NAME, "td")
            nombre_completo = celdas[0].text.strip()
            if nombre_completo == "Fecha de nacimiento:":
                nombre_completo = nombre__deportivo_jugador    
            info.append({
                "nombre__deportivo_jugador": nombre__deportivo_jugador,
                "nombre_completo": nombre_completo})
            bio["info"] = info
        except Exception as e:
            print(f"Error al obtener información de {jugador}: {e}")
        
        # Actualizar la entrada en el diccionario
        listado_jugadores[equipo][jugador] = bio
        time.sleep(1)  # Esperar para evitar sobrecargar el servidor

driver.quit()

In [ ]:
listado_jugadores

In [ ]:
# Guardamos el archivo
with open("data/historial_jugadores.pkl", "wb") as f:
    pickle.dump(listado_jugadores, f)

In [ ]:
# Abrir el archivo
with open("historial_jugadores.pkl", "rb") as f:
    listado_jugadores = pickle.load(f)

In [21]:
# Crear una URL por cada jugador
plantilla_url_jugador = "https://www.livefutbol.com/equipos/{x}"
urls_por_equipo_historico = {}
for equipo in lista_equipos:
    url = plantilla_url.format(x=equipo)
    urls_por_equipo_historico[equipo] = url

In [ ]:
urls_por_equipo_historico

In [35]:
equipos_relacionados = {}
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)
for equipo, url in urls_por_equipo_historico.items():

    driver.get(url)
    # Aceptar cookies si es necesario
    try:
        WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "cmpbntyestxt"))).click()
    except:
        pass
    # Obtener nombre completo equipo
    tabla_equipo = driver.find_element(By.CSS_SELECTOR, "table.standard_tabelle.yellow")
    filas_jugador = tabla_equipo.find_elements(By.CSS_SELECTOR, 'tr')
    celdas = filas_jugador[0].find_elements(By.TAG_NAME, "td")
    nombre_completo = celdas[0].text.strip()
    equipos_filial = {}
    historial_equipos = []
    # Obtener tabla más equipos
    tabla = driver.find_elements(By.CSS_SELECTOR, "table.standard_tabelle")
    filas_historial = tabla[2].find_elements(By.CSS_SELECTOR, 'tr')
    # Recorrer cada enlace y extraer el nombre del jugador
    for fila in filas_historial:
        celdas = fila.find_elements(By.TAG_NAME, "td")
        celdas_validas = [celdas[i].text.strip() for i in range(len(celdas)) if i % 2 == 1]
        historial_equipos.append(celdas_validas)
    equipos_filial[nombre_completo] = historial_equipos
    equipos_relacionados[equipo] = equipos_filial
    time.sleep(1)

driver.quit()

In [ ]:
equipos_relacionados

In [ ]:
# Guardamos el archivo
with open("data/equipos_relacionados.pkl", "wb") as f:
    pickle.dump(equipos_relacionados, f)

In [3]:
#Extraer equipos Pais Vasco
lista_equipos_pais_vasco_frances = []
#URLS
lista_urls = ["https://www.ceroacero.es/equipos/futbol/pais-vasco/masculino",
              "https://www.ceroacero.es/equipos/futbol/navarra/masculino",
              "https://www.ceroacero.es/equipos/futbol/aquitaine?search_txt=",
              "https://www.ceroacero.es/equipos/occitanie?search_txt="]
# Configuración del navegador
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)
# URL
for url in lista_urls:
    base_url = url
    driver.get(base_url)
    try:
        cookie_button = driver.find_element(By.CSS_SELECTOR, ".fc-button-label")
        cookie_button.click()
    except:
        pass  # En caso de que no aparezca seguir con el código
    try:
        cookie_button2 = driver.find_element(By.CSS_SELECTOR, ".ns-xhr8a-e-16")
        cookie_button2.click()
    except:
        pass  # En caso de que no aparezca seguir con el código
    ver_mas = driver.find_element(By.CSS_SELECTOR, "a#club_team_id.viewall-link").click()
    tabla = driver.find_element(By.CSS_SELECTOR, ".zz-popup-content.filter-popup-content")
    flex_items = tabla.find_elements(By.CSS_SELECTOR, "li[style*='display: flex']")
    # Recorrer cada enlace y extraer el nombre del equipo
    for item in flex_items:
        enlace = item.find_element(By.TAG_NAME, "a")
        # Obtener el texto del enlace
        nombre = enlace.text.strip()
        equipo = nombre.split("\n")[0]
        if equipo not in lista_equipos_pais_vasco_frances:
            lista_equipos_pais_vasco_frances.append(equipo)  # Añadir equipo a la lista si no está ya
        time.sleep(0.5)  # Dormir para evitar sobrecargar el servidor
driver.quit()


In [1]:
lista_equipos_pais_vasco_frances

NameError: name 'lista_equipos_pais_vasco_frances' is not defined